In [ ]:

import os
import shutil

N_TOPICS = 10
DOCUMENT_LENGTH = 100
FOLDER = "topicimg"

def vertical_topic(width, topic_index, document_length):
    """
    Generate a topic whose words form a vertical bar.
    """
    m = np.zeros((width, width))
    m[:, topic_index] = int(document_length / width)
    return m.flatten()

def horizontal_topic(width, topic_index, document_length):
    """
    Generate a topic whose words form a horizontal bar.
    """
    m = np.zeros((width, width))
    m[topic_index, :] = int(document_length / width)
    return m.flatten()

def save_document_image(filename, doc, zoom=2):
    """
    Save document as an image.
    doc must be a square matrix
    """
    height, width = doc.shape
    zoom = np.ones((width*zoom, width*zoom))
    # imsave scales pixels between 0 and 255 automatically
    sp.misc.imsave(filename, np.kron(doc, zoom))

def gen_word_distribution(ntopics, document_length):
    """
    Generate a word distribution for each of the ntopics.
    """
    width = ntopics / 2
    vsize = width ** 2
    m = np.zeros((ntopics, vsize))

    for k in range(width):
        m[k,:] = vertical_topic(width, k, document_length)

    for k in range(width):
        m[k+width,:] = horizontal_topic(width, k, document_length)

    m /= m.sum(axis=1)[:, np.newaxis] # turn counts into probabilities

    return m

def gen_document(word_dist, ntopics, vsize, length=DOCUMENT_LENGTH, alpha=0.1):
    """
    Generate a document:
    1) Sample topic proportions from the Dirichlet distribution.
    2) Sample a topic index from the Multinomial with the topic
       proportions from 1).
    3) Sample a word from the Multinomial corresponding to the topic
       index from 2).
    4) Go to 2) if need another word.
    """
    theta = np.random.mtrand.dirichlet([alpha] * ntopics)
    v = np.zeros(vsize)
    for n in range(length):
        z = index_sample(theta)
        w = index_sample(word_dist[z,:])
        v[w] += 1
    return v

def gen_documents(word_dist, ntopics, vsize, n=500):
    """
    Generate a document-term matrix.
    """
    m = np.zeros((n, vsize))
    for i in xrange(n):
        m[i, :] = gen_document(word_dist, ntopics, vsize)
    
    return m

if os.path.exists(FOLDER):
    shutil.rmtree(FOLDER)
os.mkdir(FOLDER)


In [401]:
matrix[:,1]
#test2 = word_indices(matrix[:,1])
matrix[:,1].nonzero()

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,  14.,  16.,   7.,
         0.,   0.,   9.,   0.,  16.,   2.,   0.,   0.,  13.,   0.,   4.,
         0.,   0.,  10.,   0.,   0.,   1.,   2.,   0.,   0.,   0.,   0.,
        15.,   1.,   0.,   0.,   6.,   0.,   2.,   0.,   7.,   0.,   2.,
         0.,  11.,  14.,   0.,  18.,   5.,   1.,  15.,   0.,   1.,   0.,
         0.,   0.,   3.,   0.,   1.,  18.,   0.,   3.,  22.,   0.,  12.,
         7.,   1.,  14.,   1.,   2.,   0.,   7.,   2.,  13.,   4.,   0.,
         0.,   4.,   0.,   4.,   5.,   0.,   2.,  13.,   2.,   1.,   0.,
        19.,   0.,   1.,   0.,   1.,   1.,   0.,   0.,   0.,  22.,   7.,
         0.,  16.,   0.,   1.,   3.,   0.,  17.,   1.,   0.,   2.,  10.,
         0.,  12.,   2.,   0.,   1.,   0.,   0.,  12.,   5.,   0.,   3.,
         0.,   0.,   0.,  10.,   0.,   7.,   1.,   4.,  16.,   0.,   0.,
         0.,   0.,  14.,  10.,  10.,   0.,   0.,   7.,   0.,   0.,   0.,
         0.,   2.,   0.,  12.,   0.,   2.,   1.,   

In [384]:
test2.next()

24

In [440]:

import numpy as np
import scipy as sp
from scipy.special import gammaln
import scipy.misc
import random

def index_sample(p):
    '''
    Desc: Samples from n topics distributed multinomially and returns topic number
    input: p - A one dimensional array of float64 type that contains the probability for each topic
    output: an Integer specifying which topic was chosen from a multinomial distribution
    '''
    r = random.random()
    for i in range(len(p)):
        r = r - p[i]
        if r < 0:
            return i
    return len(p) - 1

def word_indices(vec):
    '''
    Desc: Take a vector of word counts from a document and create a generator for word indices
    input: A vector from a Document Term Frequency matrix for one document.
    output: A generator object to store the word indices when called
    '''
    for idx in vec.nonzero()[0]:
        for i in xrange(int(vec[idx])):
            yield idx

def log_multi_beta(alpha, K = None):
    """
    Desc: Compute the logarithm of the multinomial beta function
    input: alpha - A vector with type float64 or a scaler of float64
           K - An integer that, if alpha is a scalar, multiplies the log by K
    output: a float64 with value of the logarithm of the multinomial beta
    """
    
    if K is None:
        return np.sum(gammaln(alpha) - gammaln(np.sum(alpha)))
    else:
        return K * gammaln(alpha) - gammaln(K * alpha)
    
class LdaSampler(object):
    
    def __init__(self, ntopics, alpha = .1, beta = .1):
        """
        Desc: Initialize values for our class object
        alpha: a float scalar
        beta: a float scalar
        ntopics: an integer for the number of topics
        """
        if not isinstance(alpha, float):
            raise Exception(" Initial value for alpha must be a floating point number (.3)")
        
        if not isinstance(beta, float):
            raise Exception(" Initial value for beta must be a floating point number (.3)")
            
        if not isinstance(ntopics, int):
            raise Exception(" The number of topics must be an integer")
            
        
        self.ntopics = ntopics
        self.alpha = alpha
        self.beta = beta
    
    def _initialize(self,matrix):
        '''
        Initialize:
        NZM: size(#Docs X #Topics) numpy array with type float 64
            The number of times document M and topic Z interact
        
        NZW: size(#Topics X #Words) numpy array with type float64
            The number of times topic Z and word W interact
        
        NM:  size(#Docs) numpy array with type float64
            Sum of documents occurances by topic and word
        
        NZ:  size(#Topics) numpy array with type float64
            Sum of Topic occurences by word and document
        
        Topics: size(?) An empty set
           Will come back to this
        '''
        ndocs, vsize = matrix.shape
        
        self.NZM = np.zeros((ndocs, self.ntopics))
        self.NZW = np.zeros((self.ntopics, vsize))
        self.NM  = np.zeros(ndocs)
        self.NZ  = np.zeros(self.ntopics)
        self.topics = {}
        
        for m in xrange(ndocs):
            # Iterates over i, doc_length - 1, and w, the size of unique_words - 1
            for i, w, in enumerate(word_indices(matrix[m,:])):
                # Initialize a random topic for each word
                z = np.random.randint(self.ntopics)
                self.NZM[m,z] += 1
                # Why is NM being +1'd for each i,w?
                self.NM[m] += 1
                self.NZW[z,w] += 1
                self.NZ[z] += 1
                self.topics[(m,i)] = z
    
    def _conditional_distribution(self, m, w):
        '''
        Desc: Compute the conditional distribution of words in document and topic
        Input: m: An integer representing the column index of the document
               w: The generator object from word_indices
        
        Output: p_z: An array size(w X 1) containing probabilities for topics of word
        '''
        vsize = self.NZW.shape[1]
        left = (self.NZW[:,w] + self.beta) / (self.NZ + self.beta * vsize)
        right = (self.NZM[m,:] + self.alpha) / (self.NM[m] + self.alpha * self.ntopics)
        p_z = abs(left * right)
        p_z /= np.sum(p_z)
        return p_z
    
    def loglikelihood(self):
        '''
        Desc: Compute the log likelihood that the model generated the data
        Input: self references
        Output: lik: float of the log likelihood
        '''
        # Why are these being repeated here?
        vsize = self.NZW.shape[1]
        ndocs = self.NZM.shape[0]
        lik = 0
        
        for z in xrange(self.ntopics):
            lik += log_multi_beta(self.NZW[z,:] + self.beta)
            lik -= log_multi_beta(self.beta, vsize)
        
        for m in xrange(ndocs):
            lik += log_multi_beta(self.NZM[m,:] + self.alpha)
            lik -= log_multi_beta(self.alpha, self.ntopics)
        
        return lik
    
    def phi_theta(self):
        '''
        Desc: Compute the probability of a word given topic
        Input: Self references
        Output: An array of size(Topic X Docs) containing the probabilities for a given topic
        '''
        num_phi = self.NZW + self.beta
        num_phi /= np.sum(num_phi, axis = 1)[:, np.newaxis]
        
        num_theta = self.NZM + self.alpha
        num_theta /= np.sum(num_theta,axis = 1)[:, np.newaxis]

        return num_phi, num_theta

        
    def run(self, matrix, maxiter = 30, burnin = 10):
        '''
        Desc: Perform Gibbs sampling for maxiter iterations
        
        Input: matrix - An array that is a Document Term Frequency Matrix
               maxiter - An integer with the number of iterations
               Burnin - TBA: An integer of the number of burnins
               
        Output: phi_theta() Two arrays, holding 
        [0] Probability of topic by word 
        [1] Probability of document by topic
        '''
        
        n_docs, vsize = matrix.shape
        self._initialize(matrix)
    
                    
        for iteration in xrange(maxiter):
            for m in xrange(n_docs):
                for i,w in enumerate(word_indices(matrix[m,:])):
                    z = self.topics[(m,i)]
                
                    # Why minus one?
                    self.NZM[m,z] -= 1
                    self.NM[m] -= 1
                    self.NZW[z,w] -= 1
                    self.NZ[z] -= 1
                  
                    p_z = self._conditional_distribution(m,w)
                    z = index_sample(p_z)
                    
                    self.NZM[m,z] += 1
                    self.NM[m] += 1
                    self.NZW[z,w] += 1
                    self.NZ[z] += 1
            
            if iteration > burnin:
                yield self.phi_theta()

                
                  
                    

In [442]:
test1 = sampler.run(matrix, maxiter = 30, burnin = 10)
test2 = test1.next()
test2[1].shape


(500, 10)

In [193]:
test2 = word_indices(matrix[:,2])


In [243]:
test2.next()

16

In [192]:
matrix[:,2]

array([  3.,   0.,   2.,   2.,   3.,   0.,   3.,  10.,   0.,   0.,   5.,
         0.,  12.,   0.,   0.,   0.,  19.,   0.,   2.,   0.,   0.,   5.,
         2.,   5.,   9.,   0.,   0.,   0.,   0.,   0.,   2.,   1.,   0.,
         0.,  19.,  15.,   0.,   1.,   0.,   3.,   0.,  12.,   0.,   0.,
         0.,   4.,  27.,   0.,  20.,   0.,  11.,   0.,   2.,   2.,   0.,
         0.,   5.,   0.,   0.,   0.,  19.,   0.,   2.,   0.,   1.,  10.,
         0.,   3.,   0.,   7.,   0.,   3.,   0.,   1.,   9.,   0.,   9.,
        22.,   4.,   1.,   6.,  14.,   1.,   2.,  21.,   0.,   0.,   0.,
         0.,   0.,   3.,   0.,   1.,  16.,   0.,   0.,   0.,  25.,   0.,
         0.,   6.,   2.,   3.,   1.,   0.,   3.,   0.,   0.,   9.,   8.,
         0.,  10.,   2.,   0.,   2.,   1.,  11.,  16.,   0.,   2.,   6.,
         4.,   0.,   0.,   4.,   0.,   7.,   3.,   5.,  11.,   0.,   0.,
         0.,   0.,   0.,  16.,  10.,   1.,   2.,   6.,   0.,   0.,   0.,
         0.,   2.,   0.,   0.,   0.,   8.,   5.,   

In [431]:
sampler = LdaSampler(N_TOPICS)
test = sampler.run(matrix, maxiter = 30, burnin = 10)

In [432]:
test1 = test.next()

In [436]:
test1[0]

array([[-0.07707858,  0.94640769, -0.17142008,  0.22437816, -0.18331186,
        -0.15457338,  0.31277376, -0.16983451,  0.08782083, -0.17815876,
        -0.14624913,  0.19742345, -0.12167278,  1.1947478 , -0.13217719,
        -0.10641165, -0.10720444, -0.12206917,  0.70183332, -0.18529383,
        -0.17776236, -0.09570905, -0.18945595, -0.14129422, -0.20570806],
       [-0.13882273, -0.14388987, -0.13189764,  0.29019508, -0.16601638,
        -0.16179377, -0.14929482, -0.16719872,  0.65925175, -0.14608563,
        -0.15334853, -0.15824677, -0.14557892, -0.09997466, -0.1411874 ,
        -0.10909552, -0.11618951, -0.14270754,  0.01015117, -0.15115277,
        -0.07531458,  0.94149143, -0.09659657,  1.2635926 ,  0.42971033],
       [-0.11008657, -0.01612684, -0.14412995,  0.30738255,  1.04836105,
        -0.17272639, -0.15560743, -0.179146  , -0.14451902, -0.11728431,
        -0.18692734, -0.12195312, -0.14354635, -0.13440327,  0.61494018,
         0.0130532 ,  0.6515125 , -0.12448205, -0

In [429]:
width = N_TOPICS / 2
vsize = width ** 2
word_dist = gen_word_distribution(N_TOPICS, DOCUMENT_LENGTH)
matrix = gen_documents(word_dist, N_TOPICS, vsize)
sampler = LdaSampler(N_TOPICS)

for it, phi_theta in enumerate(sampler.run(matrix, maxiter = 30, burnin = 10)):
    

    if it % 5 == 0:
        print "Iteration", it
        print "Likelihood", sampler.loglikelihood()
        for z in range(N_TOPICS):
            save_document_image("topicimg/topic%d-%d.png" % (it,z),
            phi[z,:].reshape(width,-1))



TypeError: list() takes at most 1 argument (2 given)

In [93]:
import nltk
import numpy as np
import pandas as pd
import os
import glob
import random

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.special import gammaln



In [95]:

DIR = r'data_folder/wordcounts'
allfiles = glob.glob(os.path.join(DIR,"*.CSV"))

p = .5
rand_sample = [ allfiles[i] for i in sorted(random.sample(xrange(len(allfiles)), int(p * len(allfiles)))) ]
rand_sample
    
np_array_list = []
for file_ in rand_sample:
    df = pd.read_csv(file_,index_col=None, header=0)
    df['source'] = file_
    np_array_list.append(df.as_matrix())
    
comb_np_array = np.vstack(np_array_list)
big_frame = pd.DataFrame(comb_np_array)
big_frame.columns = ['words','count','source']

big_frame = big_frame.pivot(index = 'source',columns = 'words', values = 'count')
big_frame = big_frame.fillna(value = 0)

In [106]:
big_frame = big_frame.loc[:, (big_frame.sum(axis = 0) > 2)]
big_frame = big_frame.loc[:, (big_frame.sum(axis = 0) < 20)]
big_frame.head()


words,aaby,aaron,aba,abab,ababab,abandon,abandoned,abba,abbott,abbreviated,...,zucker,zum,zupan,zurnal,zvi,zx,zxp,zy,zygosity,zz
source,,,,,,,,,,,,,,,,,,,,,
data_folder/wordcounts/wordcounts_10.2307_2276708.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data_folder/wordcounts/wordcounts_10.2307_2276722.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data_folder/wordcounts/wordcounts_10.2307_2276742.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data_folder/wordcounts/wordcounts_10.2307_2276818.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
data_folder/wordcounts/wordcounts_10.2307_2276825.CSV,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [104]:
big_frame.values.shape

(500, 34934)

In [100]:
for it, phi in enumerate(sampler.run(big_frame.values, maxiter = 30, burnin = 10)):
    print "Iteration", it
    print "Likelihood", sampler.loglikelihood()



Iteration 0
Likelihood -510547883685.0
Iteration 1
Likelihood -510550294175.0
Iteration 2
Likelihood -510573907380.0
Iteration 3
Likelihood -510628679589.0
Iteration 4
Likelihood -510633809299.0
Iteration 5
Likelihood -510659645673.0
Iteration 6
Likelihood -510665424219.0
Iteration 7
Likelihood -510651208299.0
Iteration 8
Likelihood -510644705702.0
Iteration 9
Likelihood -510650551356.0
Iteration 10
Likelihood -510653431940.0
Iteration 11
Likelihood -510650468845.0
Iteration 12
Likelihood -510638814662.0
Iteration 13
Likelihood -510624150253.0
Iteration 14
Likelihood -510619044655.0
Iteration 15
Likelihood -510614262016.0
Iteration 16
Likelihood -510611064296.0
Iteration 17
Likelihood -510606525451.0
Iteration 18
Likelihood -510605447122.0
Iteration 19
Likelihood -510604683425.0
Iteration 20
Likelihood -510606042584.0
Iteration 21
Likelihood -510605111489.0
Iteration 22
Likelihood -510604103147.0
Iteration 23
Likelihood -510602930467.0
Iteration 24
Likelihood -510600059783.0
Iteration 

In [108]:
sampler._conditional_distribution()

TypeError: _conditional_distribution() takes exactly 3 arguments (1 given)